In [ ]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')


fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2701, done.
remote: Counting objects: 100% (1204/1204), done.
remote: Compressing objects: 100% (370/370), done.
remote: Total 2701 (delta 939), reused 898 (delta 834), pack-reused 1497
Receiving objects: 100% (2701/2701), 2.04 MiB | 9.56 MiB/s, done.
Resolving deltas: 100% (1698/1698), done.


In [ ]:
import google.colab
!pip install -U pip
!pip install -e ColBERT/['faiss-gpu','torch']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 7.2 MB/s 

In [ ]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Trainer

In [ ]:
!unzip triples.train.small.tsv.zip -d triples.train.small.tsv
!unzip queries.train.tsv.zip -d queries.train.tsv
!unzip collection.tsv.zip -d collection.tsv

Archive:  triples.train.small.tsv.zip
  inflating: triples.train.small.tsv/triples.train.small.tsv  
  inflating: triples.train.small.tsv/__MACOSX/._triples.train.small.tsv  
Archive:  queries.train.tsv.zip
  inflating: queries.train.tsv/queries.train.tsv  
  inflating: queries.train.tsv/__MACOSX/._queries.train.tsv  
Archive:  collection.tsv.zip
  inflating: collection.tsv/collection.tsv  


In [ ]:
if __name__=='__main__':
    with Run().context(RunConfig(nranks=4, experiment="msmarco")):

        config = ColBERTConfig(
            bsize=16,
            root="exp/",
        )
        trainer = Trainer(
            triples="triples.train.small.tsv",
            queries="queries.train.tsv",
            collection="collection.tsv",
            config=config,
        )

        checkpoint_path = trainer.train()

        print(f"Saved checkpoint to {checkpoint_path}...")


In [ ]:
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [12]:
queries = 'queries.dev.tsv'
collection = 'collection.tsv'

queries = Queries(path=queries)
collection = Collection(path=collection)

f'Loaded {len(queries)} queries and {len(collection):,} passages'

[May 17, 20:43:24] #> Loading the queries from queries.dev.tsv ...
[May 17, 20:43:24] #> Got 101093 queries. All QIDs are unique.

[May 17, 20:43:24] #> Loading collection...
0M 


'Loaded 101093 queries and 16,160 passages'

In [15]:
nbits = 2   #  количество бит, используемых для кодирования каждого измерения вектора при создании индекса
doc_maxlen = 300 # обрезаем максимальную длину документов (в токенах)
max_id = 100
index_name = f'ind{nbits}bits'

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint_path, config=config)
    indexer.index(name=index_name, collection=collection[:max_id], overwrite=True)



[May 17, 21:04:30] #> Note: Output directory /content/experiments/notebook/indexes/ind2bits already exists


#> Starting...
#> Joined...


In [16]:
indexer.get_index()

'/content/experiments/notebook/indexes/ind2bits'

In [17]:
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection)

[May 17, 21:08:05] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[May 17, 21:08:05] #> Loading codec...
[May 17, 21:08:05] #> Loading IVF...
[May 17, 21:08:05] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


[May 17, 21:08:41] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1678.39it/s]

[May 17, 21:08:41] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 591.16it/s]

[May 17, 21:08:41] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


[May 17, 21:09:22] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


In [18]:
query = 'what is The Manhattan Project'
print(f"#> {query}")

# Find the top-3 passages for this query
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

#> what is The Manhattan Project

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . what is The Manhattan Project, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([ 101,    1, 2054, 2003, 1996, 7128, 2622,  102,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


	 [1] 		 28.4 		 Manhattan Project. The Manhattan Project was a research and development undertaking during World War II that produced the first nuclear weapons. It was led by the United States with the support of the United Kingdom and Canada. From 1942 to 1946, the project was under the direction of Major General Leslie Groves of the U.S. Army Corps of Engineers. Nuclear physicist Robert Oppenheimer was the director of the Los Alamos Laboratory that designed the actual bombs. The Army component of the project was designated the
	 [2] 		 27.1 		 The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to the period of the project from 194 â¦ 2-1946 under the control of the U.S. Army Corps of Engineers, under the administration of General Leslie R. Groves.
	 [3] 		 25.8 		 Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success 